In [ ]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline
import tensorflow_hub as thub
import tensorflow.keras.backend as K

In [ ]:
!pip install adversarial-robustness-toolbox
import art

     |████████████████████████████████| 1.1 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.4 MB/s 
     |████████████████████████████████| 26.3 MB 55.2 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod
from art.attacks.extraction import CopycatCNN, FunctionallyEquivalentExtraction, KnockoffNets

In [ ]:
def sample_by_class(data, labels, num_samples=100):
    sample_data = []
    sample_labels = []
    unq_labels = list(range(labels.shape[1]))
    for label in unq_labels:
        idx = labels[:,label]==1
        sample_set = data[idx][0:num_samples].copy()
        label_set = labels[idx][0:num_samples].copy()
        sample_data.append(sample_set)
        sample_labels.append(label_set)
    
    sample_data = np.concatenate(sample_data)
    sample_labels = np.concatenate(sample_labels)
    print(sample_data.shape, sample_labels.shape)
    return sample_data, sample_labels

def subset_data(data, labels, fraction=5):
    data_size = data.shape[0]
    out_size = int(data_size*fraction/100)
    idx = np.random.choice(data_size, out_size, replace=False)
    out_data = data[idx].copy()
    out_labels = labels[idx].copy()
    
    print(out_data.shape, out_labels.shape)
    return out_data, out_labels

In [ ]:
from tensorflow import keras
(x_train,y_train),(x_test,y_test) = keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
testd = x_train[0:1000].copy()
testl = y_train[0:1000].copy()

In [ ]:
def build_ganeval_model():
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
    model.add(thub.KerasLayer(ganeval_module))
    model.add(tf.keras.layers.Activation('softmax'))
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay= 1e-4),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
K.clear_session()
model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
ganeval_module = thub.Module(model_url)

In [ ]:
ge_cifar_clf = tf.keras.Sequential()
ge_cifar_clf.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
# gan_eval_model.add(gan_eval_layer)
ge_cifar_clf.add(thub.KerasLayer(ganeval_module))
ge_cifar_clf.add(tf.keras.layers.Activation('softmax'))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
ge_cifar_clf.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                       loss=tf.keras.losses.CategoricalCrossentropy(),
                       metrics=['accuracy'])
ge_cifar_clf.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 10)                7796426   
                                                                 
 activation (Activation)     (None, 10)                0         
                                                                 
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
ge_cifar_clf = build_ganeval_model()
ge_cifar_clf.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 10)                7796426   
                                                                 
 activation (Activation)     (None, 10)                0         
                                                                 
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
ge_cifar_clf.evaluate(testd, testl)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[166.622517578125, 0.11]

In [ ]:
classifier = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.4)

In [ ]:
ge_cifar_clf.evaluate(testd, testl)

[166.622517578125, 0.11]

In [ ]:
classifier = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.4)

In [ ]:
x_test_adv = attack_fgsm.generate(testd.astype(np.float32).copy())

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
ge_cifar_clf.evaluate(x_test_adv, testl)

[166.0896552734375, 0.11]

In [ ]:
def build_substitute_model(enable_logits=True):
    model = tf.keras.Sequential( )
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(32,32,3)))
    model.add(Activation("relu"))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    if enable_logits:
        model.add(Activation("linear"))
    else:
        model.add(Activation("softmax"))
        
    model.compile(loss=CategoricalCrossentropy(from_logits=enable_logits),
                  optimizer='adam',
                  metrics=['accuracy'] )
    
    return model

In [ ]:
from tensorflow.keras.layers import Conv2D
from keras.layers import Activation, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout,Flatten
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
sub_model = build_substitute_model()
sub_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_2 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                      

In [ ]:
def extraction_attack(ART_attack, x_train, y_train, fractions, epoch=30, verbose=0):
    
    data_size = x_train.shape[0]
    epochs = epoch
    
    ### Train and Test sample data from each class to get final results
    print("Extract 100 Training samples from each class:")
    x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
    print("Extract 100 Test samples from each class:")
    x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)
    
    ge_cifar_clf = build_ganeval_model()
    cloud_art_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
    
    loss1, acc1 = cloud_art_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
    loss2, acc2 = cloud_art_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
    
    clf_results = []
    
    for each_frac in fractions:
        max_queries = int(data_size*each_frac/100)
        print("Attacking the victim with %d percent of training data: %d queries..."%(each_frac, max_queries))
#         partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=each_frac)
        
        sub_clf = build_substitute_model()
        stolen_clf = KerasClassifier(model=sub_clf, clip_values=(0, 1), use_logits=True)
        
        attack = ART_attack(classifier=cloud_art_clf, 
                        batch_size_fit=32, 
                        batch_size_query=32,
                        nb_epochs=epochs,
                        nb_stolen=max_queries)
        
        stolen_clf = attack.extract(x_train, thieved_classifier=stolen_clf)

        loss3, acc3 = stolen_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
        loss4, acc4 = stolen_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
        
        clf_results.append((ART_attack.__name__, each_frac, max_queries, epochs,
                            (round(loss1,3), acc1), (round(loss2,3), acc2), 
                            (round(loss3,3), acc3), (round(loss4,3), acc4)))
        
        results_df = pd.DataFrame(clf_results, columns=['Attack', 'Fraction of data', 'Epochs','Max queries', 
                                                         'Train data-victim',
                                                       'Test data-victim', 'Train data-stolen', 
                                                        'Test data-stolen'])
    return results_df

In [ ]:
import pandas as pd
KON_results_20e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,30], epoch=5)
KON_results_30e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,30], epoch=5)
KON_results_40e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,30], epoch=5)

Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Attacking the victim with 5 percent of training data: 2500 queries...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...
Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...
Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...


In [ ]:
CCC_results_20e = extraction_attack(CopycatCNN, x_train, y_train, [5,10,30], epoch=5)
CCC_results_30e = extraction_attack(CopycatCNN, x_train, y_train, [5,10,30], epoch=5)
CCC_results_40e = extraction_attack(CopycatCNN, x_train, y_train, [5,10,30], epoch=5)

Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Attacking the victim with 5 percent of training data: 2500 queries...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Train on 2500 samples
Epoch 1/5
2500/2500 [==============================] - 1s 467us/sample - loss: 2.6844 - accuracy: 0.8912
Epoch 2/5
2500/2500 [==============================] - 1s 417us/sample - loss: 0.3684 - accuracy: 0.9172
Epoch 3/5
2500/2500 [==============================] - 1s 430us/sample - loss: 0.3338 - accuracy: 0.9188
Epoch 4/5
2500/2500 [==============================] - 1s 412us/sample - loss: 0.3361 - accuracy: 0.9188
Epoch 5/5
2500/2500 [==============================] - 1s 420us/sample - loss: 0.3048 - accuracy: 0.9188
Attacking the victim with 10 percent of training data: 5000 queries...
Train on 5000 samples
Epoch 1/5
5000/5000 [==============================] - 2s 446us/sample - loss: 1.3010 - accuracy: 0.9050
Epoch 2/5
5000/5000 [==============================] - 2s 409us/sample - loss: 0.3455 - accuracy: 0.9190
Epoch 3/5
5000/5000 [==============================] - 2s 407us/sample - loss: 0.3335 - accuracy: 0.9190
Epoch 4/5
5000/5000 [========================

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Train on 2500 samples
Epoch 1/5
2500/2500 [==============================] - 1s 492us/sample - loss: 2.7172 - accuracy: 0.9012
Epoch 2/5
2500/2500 [==============================] - 1s 406us/sample - loss: 0.3328 - accuracy: 0.9236
Epoch 3/5
2500/2500 [==============================] - 1s 401us/sample - loss: 0.3142 - accuracy: 0.9236
Epoch 4/5
2500/2500 [==============================] - 1s 414us/sample - loss: 0.3143 - accuracy: 0.9240
Epoch 5/5
2500/2500 [==============================] - 1s 407us/sample - loss: 0.3073 - accuracy: 0.9232
Attacking the victim with 10 percent of training data: 5000 queries...
Train on 5000 samples
Epoch 1/5
5000/5000 [==============================] - 2s 459us/sample - loss: 1.5474 - accuracy: 0.9118
Epoch 2/5
5000/5000 [==============================] - 2s 418us/sample - loss: 0.3436 - accuracy: 0.9232
Epoch 3/5
5000/5000 [==============================] - 2s 415us/sample - loss: 0

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Train on 2500 samples
Epoch 1/5
2500/2500 [==============================] - 1s 475us/sample - loss: 1.9794 - accuracy: 0.8960
Epoch 2/5
2500/2500 [==============================] - 1s 428us/sample - loss: 0.3617 - accuracy: 0.9204
Epoch 3/5
2500/2500 [==============================] - 1s 403us/sample - loss: 0.3651 - accuracy: 0.9204
Epoch 4/5
2500/2500 [==============================] - 1s 417us/sample - loss: 0.3458 - accuracy: 0.9204
Epoch 5/5
2500/2500 [==============================] - 1s 407us/sample - loss: 0.3357 - accuracy: 0.9204
Attacking the victim with 10 percent of training data: 5000 queries...
Train on 5000 samples
Epoch 1/5
5000/5000 [==============================] - 2s 445us/sample - loss: 1.1709 - accuracy: 0.9176
Epoch 2/5
5000/5000 [==============================] - 2s 404us/sample - loss: 0.3215 - accuracy: 0.9254
Epoch 3/5
5000/5000 [==============================] - 2s 416us/sample - loss: 0

In [ ]:
EA_results = pd.concat([KON_results_20e,KON_results_30e,KON_results_40e])
EA_results.head(20)

,Attack,Fraction of data,Epochs,Max queries,Train data-victim,Test data-victim,Train data-stolen,Test data-stolen
0,KnockoffNets,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(6.846, 0.1)","(6.814, 0.1)"
1,KnockoffNets,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(7.524, 0.1)","(7.48, 0.1)"
2,KnockoffNets,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(10.593, 0.1)","(10.475, 0.101)"
0,KnockoffNets,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(4.035, 0.1)","(4.065, 0.1)"
1,KnockoffNets,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(8.011, 0.1)","(7.948, 0.099)"
2,KnockoffNets,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(10.429, 0.1)","(10.097, 0.1)"
0,KnockoffNets,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(10.279, 0.1)","(10.339, 0.1)"
1,KnockoffNets,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(10.529, 0.1)","(10.262, 0.1)"
2,KnockoffNets,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(10.36, 0.1)","(9.957, 0.1)"


In [ ]:
EA_results = pd.concat([CCC_results_20e,CCC_results_30e,CCC_results_40e])
EA_results.head(10)

,Attack,Fraction of data,Epochs,Max queries,Train data-victim,Test data-victim,Train data-stolen,Test data-stolen
0,CopycatCNN,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(7.487, 0.1)","(7.398, 0.1)"
1,CopycatCNN,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(7.346, 0.1)","(7.29, 0.1)"
2,CopycatCNN,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(9.43, 0.1)","(9.35, 0.1)"
0,CopycatCNN,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(8.084, 0.1)","(8.097, 0.1)"
1,CopycatCNN,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(8.677, 0.1)","(8.65, 0.101)"
2,CopycatCNN,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(10.485, 0.1)","(10.215, 0.1)"
0,CopycatCNN,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(4.848, 0.1)","(4.848, 0.1)"
1,CopycatCNN,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(10.373, 0.098)","(10.175, 0.1)"
2,CopycatCNN,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(10.21, 0.1)","(10.13, 0.1)"


In [ ]:
def CCC_extraction_attack(npd_data, x_train, y_train, fractions, epoch=30, verbose=0):
    
    data_size = x_train.shape[0]
    epochs = epoch
    npd_size = npd_data.shape[0]
    
    ### Train and Test sample data from each class to get final results
    print("Extract 100 Training samples from each class:")
    x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
    print("Extract 100 Test samples from each class:")
    x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)
    
    ge_cifar_clf = build_ganeval_model()
    cloud_art_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
    
    loss1, acc1 = cloud_art_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
    loss2, acc2 = cloud_art_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
            
    sub_clf = build_substitute_model()
    stolen_clf = KerasClassifier(model=sub_clf, clip_values=(0, 1), use_logits=True)
    
    attack = CopycatCNN(classifier=cloud_art_clf, 
                        batch_size_fit=32, 
                        batch_size_query=32,
                        nb_epochs=epochs,
                        nb_stolen=npd_size)
    
    stolen_clf = attack.extract(npd_data, thieved_classifier=stolen_clf)
    
    stolen_clf._model.save_weights("CCC_temp.h5")
    
    clf_results = []
    
    for each_frac in fractions:

        max_queries = int(data_size*each_frac/100)
        print("Attacking the victim with %d percent of training data: %d queries..."%(each_frac, max_queries))
        
        stolen_clf._model.load_weights("CCC_temp.h5")
        partial_x_train, _ = subset_data(x_train, y_train, fraction=each_frac)

        stolen_clf = attack.extract(partial_x_train, thieved_classifier=stolen_clf)
        
        loss3, acc3 = stolen_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
        loss4, acc4 = stolen_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
        
        clf_results.append(("Copycat CNN", npd_size, each_frac, max_queries, epochs, 
                            (round(loss1,3), acc1), 
                            (round(loss2,3), acc2), 
                            (round(loss3,3), acc3), 
                            (round(loss4,3), acc4)))
        
        results_df = pd.DataFrame(clf_results, columns=['Attack', "NPD Queries", 'Fraction of train data', 
                                                        'Train queries', 
                                                        'Epochs trained', 'Victim clf on Train data',
                                                       'Victim clf on Test data', 'Stolen clf on Train data', 
                                                        'Stolen clf on Test data'])
    return results_df

In [ ]:
from tensorflow.keras.datasets import cifar100
(cifar100_x, cifar100_y), _ = cifar100.load_data(label_mode='fine')
print('cifar100 training data shape:', cifar100_x.shape)

# Normalize pixel values
cifar100_x = cifar100_x/255

# Sample 20,000 data points randomly from the data
idx = np.random.choice(cifar100_x.shape[0], 20000, replace=False)
cifar100_x = cifar100_x[idx].copy()
cifar100_y = cifar100_y[idx].copy()

print('Final NPD size of cifar100 training data:', cifar100_x.shape)

169017344/169001437 [==============================] - 4s 0us/step
cifar100 training data shape: (50000, 32, 32, 3)
Final NPD size of cifar100 training data: (20000, 32, 32, 3)


In [ ]:
CCC_results_20e = CCC_extraction_attack(cifar100_x, x_train, y_train, [5,10,30], epoch=10)
CCC_results_30e = CCC_extraction_attack(cifar100_x, x_train, y_train, [5,10,30], epoch=10)

Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Train on 20000 samples
Epoch 1/10
20000/20000 [==============================] - 8s 415us/sample - loss: 2.0656 - accuracy: 0.2418
Epoch 2/10
20000/20000 [==============================] - 9s 453us/sample - loss: 1.9546 - accuracy: 0.2808
Epoch 3/10
20000/20000 [==============================] - 9s 434us/sample - loss: 1.8880 - accuracy: 0.3054
Epoch 4/10
20000/20000 [==============================] - 9s 441us/sample - loss: 1.8333 - accuracy: 0.3201
Epoch 5/10
20000/20000 [==============================] - 9s 443us/sample - loss: 1.7782 - accuracy: 0.3517
Epoch 6/10
20000/20000 [==============================] - 9s 442us/sample - loss: 1.7314 - accuracy: 0.3650
Epoch 7/10
20000/20000 [==============================] - 9s 442us/sample - loss: 1.6741 - accuracy: 0.3868
Epoch 8/10
20000/20000 [==============================] - 9s 447us/sample - loss: 1.6303 - accuracy: 0.4047
Epoch 9/10
20000/20000 [==============================] - 9s 451us/sample - loss: 1.5847 - accuracy: 0.4222
Epoch

(2500, 32, 32, 3) (2500, 10)
Train on 2500 samples
Epoch 1/10
2500/2500 [==============================] - 1s 407us/sample - loss: 10.5554 - accuracy: 0.8640
Epoch 2/10
2500/2500 [==============================] - 1s 400us/sample - loss: 0.3476 - accuracy: 0.9208
Epoch 3/10
2500/2500 [==============================] - 1s 424us/sample - loss: 0.3387 - accuracy: 0.9212
Epoch 4/10
2500/2500 [==============================] - 1s 429us/sample - loss: 0.3356 - accuracy: 0.9208
Epoch 5/10
2500/2500 [==============================] - 1s 410us/sample - loss: 0.3367 - accuracy: 0.9212
Epoch 6/10
2500/2500 [==============================] - 1s 413us/sample - loss: 0.3367 - accuracy: 0.9204
Epoch 7/10
2500/2500 [==============================] - 1s 421us/sample - loss: 0.3288 - accuracy: 0.9216
Epoch 8/10
2500/2500 [==============================] - 1s 436us/sample - loss: 0.3334 - accuracy: 0.9212
Epoch 9/10
2500/2500 [==============================] - 1s 434us/sample - loss: 0.3301 - accuracy: 0

Attacking the victim with 10 percent of training data: 5000 queries...
(5000, 32, 32, 3) (5000, 10)
Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 2s 411us/sample - loss: 5.3689 - accuracy: 0.8828
Epoch 2/10
5000/5000 [==============================] - 2s 414us/sample - loss: 0.3371 - accuracy: 0.9280
Epoch 3/10
5000/5000 [==============================] - 2s 414us/sample - loss: 0.3328 - accuracy: 0.9268
Epoch 4/10
5000/5000 [==============================] - 2s 421us/sample - loss: 0.3139 - accuracy: 0.9286
Epoch 5/10
5000/5000 [==============================] - 2s 423us/sample - loss: 0.3191 - accuracy: 0.9280
Epoch 6/10
5000/5000 [==============================] - 2s 411us/sample - loss: 0.3117 - accuracy: 0.9286
Epoch 7/10
5000/5000 [==============================] - 2s 424us/sample - loss: 0.3131 - accuracy: 0.9284
Epoch 8/10
5000/5000 [==============================] - 2s 418us/sample - loss: 0.3164 - accuracy: 0.9276
Epoch 9/10
5000/5000 [========

...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Train on 20000 samples
Epoch 1/10
20000/20000 [==============================] - 8s 410us/sample - loss: 2.0543 - accuracy: 0.2411
Epoch 2/10
20000/20000 [==============================] - 8s 400us/sample - loss: 1.9399 - accuracy: 0.2865
Epoch 3/10
20000/20000 [==============================] - 8s 405us/sample - loss: 1.8769 - accuracy: 0.3092
Epoch 4/10
20000/20000 [==============================] - 8s 412us/sample - loss: 1.8220 - accuracy: 0.3263
Epoch 5/10
20000/20000 [==============================] - 9s 439us/sample - loss: 1.7681 - accuracy: 0.3487
Epoch 6/10
20000/20000 [==============================] - 9s 441us/sample - loss: 1.7085 - accuracy: 0.3728
Epoch 7/10
20000/20000 [==============================] - 9s 432us/sample - loss: 1.6645 - accuracy: 0.3852
Epoch 8/10
20000/20000 [==============================] - 9s 430us/sample - loss: 1.6170 - accuracy: 0.4076
Epoch 9/10
20000/20000 [==============================] - 9s 431us/sample - loss: 1.5708 - accuracy: 0.4255
Epoch

(2500, 32, 32, 3) (2500, 10)
Train on 2500 samples
Epoch 1/10
2500/2500 [==============================] - 1s 404us/sample - loss: 6.0484 - accuracy: 0.8672
Epoch 2/10
2500/2500 [==============================] - 1s 418us/sample - loss: 0.3102 - accuracy: 0.9336
Epoch 3/10
2500/2500 [==============================] - 1s 416us/sample - loss: 0.2995 - accuracy: 0.9344
Epoch 4/10
2500/2500 [==============================] - 1s 413us/sample - loss: 0.2956 - accuracy: 0.9344
Epoch 5/10
2500/2500 [==============================] - 1s 417us/sample - loss: 0.2969 - accuracy: 0.9344
Epoch 6/10
2500/2500 [==============================] - 1s 430us/sample - loss: 0.2911 - accuracy: 0.9348
Epoch 7/10
2500/2500 [==============================] - 1s 419us/sample - loss: 0.2923 - accuracy: 0.9344
Epoch 8/10
2500/2500 [==============================] - 1s 425us/sample - loss: 0.2946 - accuracy: 0.9340
Epoch 9/10
2500/2500 [==============================] - 1s 434us/sample - loss: 0.2948 - accuracy: 0.

Attacking the victim with 10 percent of training data: 5000 queries...
(5000, 32, 32, 3) (5000, 10)
Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 2s 396us/sample - loss: 5.0084 - accuracy: 0.8638
Epoch 2/10
5000/5000 [==============================] - 2s 398us/sample - loss: 0.4287 - accuracy: 0.9178
Epoch 3/10
5000/5000 [==============================] - 2s 407us/sample - loss: 0.3557 - accuracy: 0.9226
Epoch 4/10
5000/5000 [==============================] - 2s 414us/sample - loss: 0.3371 - accuracy: 0.9230
Epoch 5/10
5000/5000 [==============================] - 2s 408us/sample - loss: 0.3297 - accuracy: 0.9236
Epoch 6/10
5000/5000 [==============================] - 2s 403us/sample - loss: 0.3198 - accuracy: 0.9244
Epoch 7/10
5000/5000 [==============================] - 2s 405us/sample - loss: 0.3181 - accuracy: 0.9244
Epoch 8/10
5000/5000 [==============================] - 2s 403us/sample - loss: 0.3094 - accuracy: 0.9244
Epoch 9/10
5000/5000 [========

Attacking the victim with 30 percent of training data: 15000 queries...
(15000, 32, 32, 3) (15000, 10)
Train on 15000 samples
Epoch 1/10
15000/15000 [==============================] - 6s 402us/sample - loss: 1.8693 - accuracy: 0.9032
Epoch 2/10
15000/15000 [==============================] - 6s 397us/sample - loss: 0.3234 - accuracy: 0.9229
Epoch 3/10
15000/15000 [==============================] - 6s 405us/sample - loss: 0.3204 - accuracy: 0.9230
Epoch 4/10
15000/15000 [==============================] - 6s 421us/sample - loss: 0.3146 - accuracy: 0.9229
Epoch 5/10
15000/15000 [==============================] - 6s 432us/sample - loss: 0.3107 - accuracy: 0.9229
Epoch 6/10
15000/15000 [==============================] - 6s 433us/sample - loss: 0.3100 - accuracy: 0.9231
Epoch 7/10
15000/15000 [==============================] - 6s 423us/sample - loss: 0.3045 - accuracy: 0.9234
Epoch 8/10
15000/15000 [==============================] - 6s 424us/sample - loss: 0.3094 - accuracy: 0.9232
Epoch 9/10

In [ ]:
CCC_results = pd.concat([CCC_results_20e,CCC_results_30e])
CCC_results.head(24)

,Attack,NPD Queries,Fraction of train data,Train queries,Epochs trained,Victim clf on Train data,Victim clf on Test data,Stolen clf on Train data,Stolen clf on Test data
0,Copycat CNN,20000,5,2500,10,"(170.88, 0.098)","(170.812, 0.101)","(7.585, 0.1)","(7.502, 0.1)"
1,Copycat CNN,20000,10,5000,10,"(170.88, 0.098)","(170.812, 0.101)","(7.932, 0.1)","(7.928, 0.1)"
2,Copycat CNN,20000,30,15000,10,"(170.88, 0.098)","(170.812, 0.101)","(8.233, 0.1)","(8.234, 0.1)"
0,Copycat CNN,20000,5,2500,10,"(170.88, 0.098)","(170.812, 0.101)","(7.113, 0.1)","(7.098, 0.099)"
1,Copycat CNN,20000,10,5000,10,"(170.88, 0.098)","(170.812, 0.101)","(13.081, 0.1)","(12.647, 0.1)"
2,Copycat CNN,20000,30,15000,10,"(170.88, 0.098)","(170.812, 0.101)","(11.589, 0.101)","(11.382, 0.1)"


In [ ]:
x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)

(1000, 32, 32, 3) (1000, 10)
(1000, 32, 32, 3) (1000, 10)


In [ ]:
partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=0.5)

partial_x_train = partial_x_train.reshape(-1,32*32*3)

### Tried even with standardizing the values around '0'
x_mean = partial_x_train.mean()
x_std = partial_x_train.std()
# partial_x_train = (partial_x_train - x_mean)/x_std

partial_x_train.shape

(250, 32, 32, 3) (250, 10)


(250, 3072)

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense
fen_eval_model = Sequential()
fen_eval_model.add(Dense(1024, activation="relu", input_shape=(32*32*3,)))
fen_eval_model.add(Dense(10, activation="linear"))

fen_eval_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001,),
    metrics=["accuracy"],
)

In [ ]:
fen_eval_model.fit(x_train.reshape(-1,32*32*3), y_train, batch_size=32, epochs=10, verbose=1)

Train on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 8s 160us/sample - loss: 31.5414 - accuracy: 0.2478
Epoch 2/10
50000/50000 [==============================] - 8s 158us/sample - loss: 10.8145 - accuracy: 0.2768
Epoch 3/10
50000/50000 [==============================] - 8s 159us/sample - loss: 5.9175 - accuracy: 0.2820
Epoch 4/10
50000/50000 [==============================] - 8s 156us/sample - loss: 2.5733 - accuracy: 0.2193
Epoch 5/10
50000/50000 [==============================] - 8s 158us/sample - loss: 2.0365 - accuracy: 0.2638
Epoch 6/10
50000/50000 [==============================] - 8s 155us/sample - loss: 1.8896 - accuracy: 0.3176
Epoch 7/10
50000/50000 [==============================] - 8s 157us/sample - loss: 1.8439 - accuracy: 0.3337
Epoch 8/10
50000/50000 [==============================] - 8s 157us/sample - loss: 1.8216 - accuracy: 0.3428
Epoch 9/10
50000/50000 [==============================] - 8s 157us/sample - loss: 1.8012 - accuracy: 0.3503
Epo

In [ ]:
fen_eval_model.evaluate(x_test.reshape(-1,32*32*3), y_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[1.7768745347976684, 0.3667]

In [ ]:
fen_eval_model.evaluate(partial_x_train, partial_y_train)

[1.7442967567443848, 0.336]

In [ ]:
flat_art_clf = KerasClassifier(model=fen_eval_model, clip_values=(0, 1), use_logits=False)

attack_FEN = FunctionallyEquivalentExtraction(classifier=flat_art_clf, num_neurons=1024)
stolen_clf_FEN = attack_FEN.extract(partial_x_train, partial_y_train,
                                    delta_0=0.05,
                                    fraction_true=0.2,
                                    rel_diff_slope=0.1,
                                    rel_diff_value=0.1,
                                    delta_init_value=0.1,
                                    delta_value_max=2,
                                    d2_min=0.04,
                                    d_step=0.01,
                                    delta_sign=0.2,
                                    unit_vector_scale=1000)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.7/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:221: RuntimeWarning: divide by zero encountered in true_divide
  if np.sum(np.abs((m_1 - m_2) / m_1) < rel_diff_slope) > fraction_true * self.num_classes:
/usr/local/lib/python3.7/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:225: RuntimeWarning: divide by zero encountered in true_divide
  t_hat = t_1 + np.divide(y_2 - y_1 - (t_2 - t_1) * m_2, m_1 - m_2)
/usr/local/lib/python3.7/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:226: RuntimeWarning: divide by zero encountered in true_divide
  y_hat = y_1 + m_1 * np.divide(y_2 - y_1 - (t_2 - t_1) * m_2, m_1 - m_2)
/usr/local

KeyboardInterrupt: ignored

In [ ]:
def build_ganeval_model_flat():
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = Sequential()
#     model.add(InputLayer(input_shape=(32*32*3,)))
    model.add(Reshape((32,32,3),input_shape=(32*32*3,)))
    model.add(thub.KerasLayer(ganeval_module))
    model.add(Activation('relu'))
        
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                  loss=CategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

In [ ]:
ge_flat_clf = build_ganeval_model_flat()
ge_flat_clf.summary()

In [ ]:
ge_flat_clf.evaluate(partial_x_train, partial_y_train)

In [ ]:
flat_art_clf = KerasClassifier(model=ge_flat_clf, clip_values=(0, 1), use_logits=False)

attack_FEN = FunctionallyEquivalentExtraction(classifier=flat_art_clf, num_neurons=100)
stolen_clf_FEN = attack_FEN.extract(partial_x_train, partial_y_train,
                                    delta_0=0.05,
                                    fraction_true=0.3,
                                    rel_diff_slope=0.1,
                                    rel_diff_value=0.1,
                                    delta_init_value=0.1,
                                    delta_value_max=2,
                                    d2_min=0.04,
                                    d_step=0.01,
                                    delta_sign=0.2,
                                    unit_vector_scale=1000)